In [5]:
from dotenv import load_dotenv
import os
import requests
import datetime as dt
import psycopg

load_dotenv()

True

In [6]:
def get_weather_for_city(city):
    api_key = os.getenv("WEATHER_APIKEY")
    endpoint = f"http://api.openweathermap.org/data/2.5/weather?units=metric&appid={api_key}&q={city}"
    response = requests.get(endpoint)
    if response.status_code == 200:
        response_json = response.json()
        weather_date = response_json["dt"]
        weather_city = response_json["name"]
        weather_temp = response_json["main"]["temp"]
        weather_feels = response_json["main"]["feels_like"]
        weather_description = response_json["weather"][0]["description"]
        return [weather_date, weather_city, weather_temp, weather_feels, weather_description]

In [7]:
def formatDate(data):
    data[0] = dt.datetime.fromtimestamp(data[0])
    return data

In [8]:
def populate_database(data):
    dbconn = os.getenv("DBCONN")
    conn = psycopg.connect(dbconn)
    cur = conn.cursor()
    
    data = [formatDate(x) for x in data if x != None]
    
    cur.executemany(
        '''
            INSERT INTO weather_data(date, city, temp, feels, description)
            VALUES (%s, %s, %s, %s, %s);
        ''', 
        data
    )
    conn.commit()
    cur.close()
    conn.close()

In [9]:
cities = ["Berlin", "Sydney", "Tokyo"]
results = []

for city in cities:
    results.append(get_weather_for_city(city))

populate_database(results)